In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/weather-prediction/weather_prediction_dataset.csv
/kaggle/input/weather-prediction/metadata.txt
/kaggle/input/weather-prediction/weather_prediction_bbq_labels.csv


In this notebook, we use Spark Machine Learning library to work on this dataset. A related notebook that explored this dataset using ML techniques from sklearn and ANN/CNN from tensorflow can be found here:

https://www.kaggle.com/code/lorresprz/ann-cnn-randomforest-predicting-nice-weather



In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=a0f4973008ddc1dbc317104f9b68140a9df77dbb4ee2e0281a8e0735658fc42e
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('weather_pred').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/08 06:32:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df1 = spark.read.csv('/kaggle/input/weather-prediction/weather_prediction_dataset.csv', header=True, inferSchema=True)
df2 = spark.read.csv('/kaggle/input/weather-prediction/weather_prediction_bbq_labels.csv',header=True, inferSchema=True)

There are 165 columns for the 18 cities in Europe. For each cities, multiple measurements including wind gust, wind speed, cloud cover, humidity, pressure, global radiation, precipitation, sunshine, minimum temperature, maximum temperature were recorded. Furthermore, the set of measurements varies from city to city (for example, some city might have wind speed in place of cloud cover or some might have wind gust in place of wind speed). For this notebook, we will focus on Dresden, a city located on the eastern side of Germany. The objective of this is to predict whether the weather is okay for to have a barbecue outdoors.

In [6]:
df1.printSchema()

root
 |-- DATE: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- BASEL_cloud_cover: integer (nullable = true)
 |-- BASEL_humidity: double (nullable = true)
 |-- BASEL_pressure: double (nullable = true)
 |-- BASEL_global_radiation: double (nullable = true)
 |-- BASEL_precipitation: double (nullable = true)
 |-- BASEL_sunshine: double (nullable = true)
 |-- BASEL_temp_mean: double (nullable = true)
 |-- BASEL_temp_min: double (nullable = true)
 |-- BASEL_temp_max: double (nullable = true)
 |-- BUDAPEST_cloud_cover: integer (nullable = true)
 |-- BUDAPEST_humidity: double (nullable = true)
 |-- BUDAPEST_pressure: double (nullable = true)
 |-- BUDAPEST_global_radiation: double (nullable = true)
 |-- BUDAPEST_precipitation: double (nullable = true)
 |-- BUDAPEST_sunshine: double (nullable = true)
 |-- BUDAPEST_temp_mean: double (nullable = true)
 |-- BUDAPEST_temp_max: double (nullable = true)
 |-- DE_BILT_cloud_cover: integer (nullable = true)
 |-- DE_BILT_wind_speed: d

In [7]:
df2.printSchema()

root
 |-- DATE: integer (nullable = true)
 |-- BASEL_BBQ_weather: boolean (nullable = true)
 |-- BUDAPEST_BBQ_weather: boolean (nullable = true)
 |-- DE_BBQ_weather: boolean (nullable = true)
 |-- DRESDEN_BBQ_weather: boolean (nullable = true)
 |-- DUSSELDORF_BBQ_weather: boolean (nullable = true)
 |-- HEATHROW_BBQ_weather: boolean (nullable = true)
 |-- KASSEL_BBQ_weather: boolean (nullable = true)
 |-- LJUBLJANA_BBQ_weather: boolean (nullable = true)
 |-- MAASTRICHT_BBQ_weather: boolean (nullable = true)
 |-- MALMO_BBQ_weather: boolean (nullable = true)
 |-- MONTELIMAR_BBQ_weather: boolean (nullable = true)
 |-- MUENCHEN_BBQ_weather: boolean (nullable = true)
 |-- OSLO_BBQ_weather: boolean (nullable = true)
 |-- PERPIGNAN_BBQ_weather: boolean (nullable = true)
 |-- SONNBLICK_BBQ_weather: boolean (nullable = true)
 |-- STOCKHOLM_BBQ_weather: boolean (nullable = true)
 |-- TOURS_BBQ_weather: boolean (nullable = true)

